In [ ]:
import os
import sys
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import scipy.signal as signal
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from tqdm import tqdm

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from paths import path_samoc, path_results, path_CESM, path_prace
from paths import path_ocn_ctrl, path_ocn_rect_ctrl
from paths import path_ocn_rcp,  path_ocn_rect_rcp
from timeseries import IterateOutputCESM

# averaging timeseries and creating new netCDF files

In [ ]:
from aa_derivation_fields import DeriveField
from ab_derivation_SST import DeriveSST as DS

In [ ]:
DS.generate_monthly_SST_files(run='ctrl')

In [ ]:
da = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ctrl.nc')

In [ ]:
da.time

In [ ]:
DeriveField('ctrl').yrly_avg_nc(domain='atm', fields=['T','T850','U','V'], test=False)

In [ ]:
for y, m, s in IterateOutputCESM(domain='ocn_rect', run='ctrl', tavg='monthly'):
    if m==1: print(y, s)

In [ ]:
# DS.generate_yrly_SST_files('ctrl')

# Statistical testing

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)
AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)

In [ ]:
X1 = SST_yrly_detr_ctrl[:, 1000,1000]
X2 = SST_yrly_detr_ctrl[:, 2000,1000]
X3 = SST_yrly_detr_ctrl[:, 1000,2000]

In [ ]:
X1.plot()
X2.plot()
X3.plot()

In [ ]:
def batch_act(X, m):
    """estimating autocorrelation time from batch method 
    X .. time series
    m .. number of batches
    """
    n = int(np.floor(len(X)/m))
    tau = 0
    for i in range(m):
        sm   = np.var(X[i*n:(i+1)*n]) 
        s    = np.var(X)
        tau += sm**2/s**2
#         print(tau)
    return tau

In [ ]:
for i in np.arange(1, 20):
    print(f'{i:2d}: {batch_act(X1.values, i):5.2f}, {batch_act(X2.values, i):5.2f}, {batch_act(X3.values, i):5.2f}')



In [ ]:
AMO_ctrl.plot()
chebychev(AMO_ctrl, 13).plot()


In [ ]:
def auto_correlate(X):
#     X_ac = np.correlate(X, X, mode='full')[len(X):]
#     print(np.var(X))
#     X_ac = (np.correlate(X, X, mode='full')/(np.var(X)))[len(X):]
    X_ac = np.zeros((len(X)-1))
    for t in np.arange(1,len(X)-1):
        X_shifted = np.zeros((len(X)))
        X_shifted[t:] = X[:-t]
        X_ac[t-1] = np.corrcoef(X, X_shifted)[0,1]
    tau_X = 1 + 2*np.sum(X_ac)
    return X_ac, tau_X

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
plt.plot(auto_correlate(X)[0])

## effective sample size

In [ ]:
n = 1000
for i in range(20):
    # np.random.seed(seed=42)
    X = np.random.normal(size=(n))
    X = X-X.mean()
    X10 = running_mean(X, 100)
    # X10 = X10-X10.mean()
#     plt.plot(X)
    # plt.plot(lowpass(X, 100)[49:-50])
#     plt.plot(np.arange(49,n-50), X10)

    X_ac, tau_X = auto_correlate(X)
    X10_ac, tau_X10 = auto_correlate(X10)
    print(i, n, int(n/tau_X), int(n/tau_X10))

In [ ]:
plt.hist(X)

In [ ]:
X_ac, tau_X = auto_correlate(X)
X10_ac, tau_X10 = auto_correlate(X10)

In [ ]:
print(X_ac.sum(),X10_ac.sum())

In [ ]:
plt.axhline(0)
plt.plot(X_ac[49:-50])
# plt.plot(X10_ac)

In [ ]:
print(n, n/tau_X, n/tau_X10)

In [ ]:
plt.scatter(X[49:-50], X10)
np.corrcoef(X[49:-50], X10)